This challenge implements a KMOV cryptosystem based on twisted Edwards curve (see [[1]](#1)), the vulnerablity comes from the generation of the public exponent $e$ parameter, here $e$ satisfies the equation:

$$ex+(p + 1)(q + 1)y = z$$

where $x$, $y$ and $z$ satisfy:

$$xy < \frac{\sqrt{2N}}{12} \quad {\rm and} \quad \left|z \right| < \frac{(p - q)N^{0.21}y}{3(p + q)}$$

In this case, we can use Diophantine approximations to find $x$ and $y$ among the convergents of the continued fraction expansion of $\frac{e}{N}$ (see [[2]](#2), Theorem 1, 2), after finding $x$ and $y$, we can get an approximation $\tilde{p}$ of $p$ satisfying $|p - \tilde{p}| < N^{0.21}$, which leads to the factorization of $N$ by using Coppersmith's method for finding small roots of modular polynomial equations (see [[2]](#2), Theorem 4).

After that, we can get the private key of this challenge, the only thing left is to figure out that the encryption process is just computing a scalar multiplication on a twisted Edwards curve, since we have already got the private key, we can just use it to decrypt the ciphertext, however, the method used here to compute scalar multiplication is through repeated addition, this is a fully exponential approach and it runs too slow, so that we can simply use the double-and-add algorithm to speed up this process.

Here is my final solver:

In [1]:
#!/usr/bin/env sage

def on_curve(C, P):
    a, d, p = C
    x, y = P
    return (a*x**2 + y**2 - d*x**2*y**2) % p == 1

def point_add(C, P, Q):
    a, d, p = C
    x1, y1 = P
    x2, y2 = Q
    assert on_curve(C, P) and on_curve(C, Q)
    x3 = (x1 * y2 + y1 * x2) * inverse_mod(1 + d * x1 * x2 * y1 * y2, p) % p
    y3 = (y1 * y2 - a * x1 * x2) * inverse_mod(1 - d * x1 * x2 * y1 * y2, p) % p
    return (int(x3), int(y3))

def point_mul(C, P, s):
    Q = (0, 1)
    while s > 0:
        if s % 2 == 1:
            Q = point_add(C, Q, P)
        P = point_add(C, P, P)
        s //= 2
    return Q

bits = 1024

'''
f = open('output.txt', 'rb').read()
data = f.replace(b'(', b'').replace(b')', b'').split(b'\n')

e, N = tuple(map(int, data[0].split(b', ')))
ct = tuple(map(int, data[1].split(b', ')))
'''

e, N = (288969517294013178236187423377607850772706067194956328319540958788120421760563745859661120809993097599452236235703456953461446476016483100948287481999230043898368061651387268308645842547879026821842863879967704742559469599469159759360184157244907772315674219466971226019794131421405331578417729612598931842872757269134756215101980595515566901217084629217607502582265295755863799167702741408881294579819035951888562668951997777236828957162036234849207438819692480197365737237130918496390340939168630111890207700776894851839829623749822549994705192645373973493114436603297829506747411555800330860323339168875710029679, 6321130275268755691320586594611921079666212146561948694592313061609721619539590734495630218941969050343046016393977582794839173726817429324685098585960482266998399162720208269336303520478867387042992449850962809825380612709067651432344409349798118550026702892042869238047094344883994914342037831757447770321791092478847580639207346027164495372017699282907858775577530313354865815011726710796887715414931577176850854690237886239119894136091932619828539390021389626283175740389396541552356118540397518601098858527880603493380691706649684470530670258670128352699647582718206243920566184954440517665446820063779925391893)
ct = (5899152272551058285195694254667877221970753694584926104666866605696215068207480540407327508300257676391022109169902014292744666257465490629821382573289737174334198164333033128913955350103258256280828114875165476209826215601196920761915628274301746678705023551051091500407363159529055081261677043206130866838451325794109635288399010815200512702451748093168790121961904783034526572263126354004237323724559882241164587153748688219172626902108911587291552030335170336301818195688699255375043513696525422124055880380071075595317183172843771015029292369558240259547938684717895057447152729328016698107789678823563841271755, 253027286530960212859400305369275200777004645361154014614791278682230897619117833798134983197915876185668102195590667437488411251835330785944874517235915807926715611143830896296709467978143690346677123639363900536537534596995622179904587739684155397043547262126131676366948937690378306959846311626889534352806134472610026603322329394769864728875293696851590640974817297099985799243285824842399573006841275494668451690794643886677303573329060084436896592291515021246248961538322485059619863786362159459122242131918702862396595818404578595841492379025543989260901540257216728185425462070297720884398220421012139424567)

res = [(i.denom(), i.numer()) for i in continued_fraction(e / N).convergents()]

P.<pp> = PolynomialRing(Zmod(N))

for x, y in res:
    if Integer(y).nbits() in range(bits // 2 - 8, bits // 2) and Integer(x).nbits() in range(bits // 2 - 8, bits // 2):
        U = (e * x // y) - N - 1
        V = int(sqrt(abs(U**2 - 4 * N)))
        p_0 = (U+V) // 2
        f = ((p_0 << 576) >> 576) + pp
        r = f.small_roots(X = 2**(bits - 576), beta = 0.4)
        if r != []:
            p = int(p_0 + r[0])
            if (N % p == 0) and is_prime(p):
                break

q = N // p
k = inverse_mod(e, (p + 1) * (q + 1))

d = (((ct[1])**2 - 1) * inverse_mod(((ct[1])**2 + 1) * (ct[0])**2, N)) % N
pt = point_mul((-d, d, N), ct, k)
flag = pt[0].to_bytes(32, 'big') + pt[1].to_bytes(32, 'big')
print(flag)

b'X-NUCA{Youve_Forg0tt3n_th3_t4ste_0f_Rea1_h0ney_6f36940f714710af}'


**P.S.**

* According to ([[2]](#2), Theorem 3), suppose we know an approximation $\tilde{p}$ of $p$ with $|p - \tilde{p}| < N^{0.25}$, then $N$ can be factored in polynomial time, this means for a 2048-bit $N$, given 512 high order bits of $p$ is encough to factorize $N$, but in practice, if you want to use `.small_roots()` method in SageMath to get the result, usually you need 576 bits known to get the result quickly, you can also reduce the number of bits required by reducing the value of the `epsilon` parameter in `.small_roots()` (e.g., 540 bits known with `epsilon = 0.02` and 530 bits with `epsilon = 0.01` worked well), but it will cost more time.

* Also, some new improved attacks on the KMOV cryptosystem have been proposed, these attacks work when the private key is suitably small and the new results improve the former attacks on the KMOV cryptosystem, see ([[3]](#3), Section 3, 4).

* The content of the FLAG is a quote from movie *Scent of a Woman* "You're so wrapped up in sugar, you've forgotten the taste of real honey!"

**References**

<a id="1" href="https://eprint.iacr.org/2019/1051.pdf">[1] Boudabra, M., Nitaj, A. A new public key cryptosystem based on Edwards curves. J. Appl. Math. Comput. 61, 431–450 (2019).</a>

<a id="2" href="https://eprint.iacr.org/2011/427.pdf">[2] A. Nitaj, A new attack on the KMOV cryptosystem, Bulletin of the Korean Math- ematical Society 51 (5), 1347–1356, 2014.</a>

<a id="3" href="https://eprint.iacr.org/2019/1052.pdf">[3] Nitaj, Abderrahmane, Willy Susilo, and Joseph Tonien. "Improved Cryptanalysis of the KMOV Elliptic Curve Cryptosystem." International Conference on Provable Security. Springer, Cham, 2019.</a>